## 경기도 보도자료 크롤링

In [1]:
import requests
import lxml.html
import pandas as pd
import sqlite3
from pandas.io import sql
import os

In [2]:
#DB에 저장하기
def db_save(KYONGGI_LIST):
    with sqlite3.connect(os.path.join('.','sqliteDB')) as con: # sqlite DB 파일이 존재하지 않는 경우 파일생성
        try:
            KYONGGI_LIST.to_sql(name = 'KYONGGI_LIST', con = con, index = False, if_exists='append') 
            #if_exists : {'fail', 'replace', 'append'} default : fail
        except Exception as e:
            print(str(e))
        print(len(KYONGGI_LIST), '건 저장완료..')

In [8]:
page = 10  # 시작 페이지
end_page = 15 # 끝날 페이지

while(True):
    df_list = []
    response = requests.get('https://gnews.gg.go.kr/briefing/brief_gongbo.do?page={}&BS_CODE=s017&period_1=&period_2=&search=0&keyword=&subject_Code=BO01'\
                            .format(page))
    root = lxml.html.fromstring(response.content)
    for tr in root.xpath('//*[@id="chk-table"]/tbody/tr'):
            a = tr.xpath('td[2]/a')[0] 
            url = ('https://gnews.gg.go.kr/' + a.get('href'))
            dep = tr.xpath('td[3]')[0]
            date = tr.xpath('td[5]')[0]
            view =  tr.xpath('td[6]')[0]
            df_list.append(
                pd.DataFrame({
 
                 'url' : [url],
                 'title' : [a.text],
                 'dept'  :  [dep.text],
                 'reg_date' : [date.text.strip()],
                 'view' : [view.text],    
            })
            )
    
    df_10 = pd.concat(df_list)
    
    db_save(df_10)
    

    if  page >= end_page:
        break;
    else:
        page = page + 1
    

10 건 저장완료..
10 건 저장완료..


In [9]:
# 데이터 출력
def db_select():
    with sqlite3.connect(os.path.join('.','sqliteDB')) as con: # sqlite DB 파일이 존재하지 않는 경우 파일생성
        try:
            query = 'SELECT * FROM KYONGGI_LIST'
            KYONGGI_LIST = pd.read_sql(query, con = con)
        except Exception as e:
            print(str(e)) 
        return KYONGGI_LIST
db_select()    

In [7]:
# DB데이터 삭제
def db_delete():
    with sqlite3.connect(os.path.join('.','sqliteDB')) as con: # sqlite DB 파일이 존재하지 않는 경우 파일생성
        try:
            cur = con.cursor()
            sql = 'DELETE FROM KYONGGI_LIST'
            cur.execute(sql)
        except Exception as e:
            print(str(e))
db_delete()

In [ ]:
#DB 삭제
def db_DROP():
    with sqlite3.connect(os.path.join('.','sqliteDB')) as con: # sqlite DB 파일이 존재하지 않는 경우 파일생성
        try:
            cur = con.cursor()
            sql = 'DROP TABLE KYONGGI_LIST'
            cur.execute(sql)
        except Exception as e:
            print(str(e))
db_DROP()          

In [ ]:
#EXCEL에 저장
KYONGGI_LIST = db_select()

def save_excel(KYONGGI_LIST):
    excel = pd.ExcelWriter('경기도보도자료.xlsx')
    KYONGGI_LIST.to_excel(excel,'.',index=False)
    excel.save()
    
save_excel(KYONGGI_LIST)